# Void Network Topology: Pe on Social Graphs

**The question:** When platforms (or agents) are coupled by recommendation, cross-posting,
or social pressure, does network topology change aggregate Pe?

nb14 derived the first inter-agent coupling constant $J_{\rm eff}^* = -0.520$ for
counter-cyclical stablecoin holders. This notebook generalises to **N-agent networks**
on arbitrary graphs.

**Setting:**
- $N$ agents, each with constraint level $c_i$ (→ single-agent Pe_i)
- Edge weight $J_{ij}$ = coupling strength (positive = ferromagnetic/aligning,
  negative = antiferromagnetic/counter-cyclical)
- Self-consistent mean-field: $\theta_i = \sigma\!\bigl(2b_{{\rm net},i} + 2J\sum_j A_{ij}\theta_j\bigr)$
- Network Pe$_i = K\sinh\!\bigl(2(b_{{\rm net},i} + J\sum_j A_{ij}\theta_j)\bigr)$

**Four graph topologies:** Complete, Erdős-Rényi (random), ring (local), Barabási-Albert (scale-free).

**Three experiments:**
1. **Topology comparison** — same $c$, same $J$, 4 topologies: how much does topology inflate Pe?
2. **Hub contamination** — one deep-void hub connected to constrained peripheral nodes.
   How much does one void platform pull the network above Pe = 1?
3. **Percolation** — Pe$_{\rm net}$ vs edge density $p$ (Erdős-Rényi).
   Where is the connectivity threshold for significant void amplification?

**Key finding (preview):** Scale-free networks amplify Pe super-linearly via hubs.
A single deep-void hub (Pe ≈ 22) can drag a network of otherwise-compliant nodes
(Pe < 1) above the Pe = 1 threshold through ferromagnetic coupling alone.

**Relates to:** `14_two_agent_contamination_critical.ipynb`, `10_cross_domain_calibration.ipynb`,
`22_regulatory_intervention_optimum.ipynb`. No THRML sampler needed — mean-field is exact in
the thermodynamic limit and sufficient for topology discrimination at finite N.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.special import expit
from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix

# Canonical THRML parameters
b_alpha = 0.5 * np.log(0.85 / 0.15)
b_gamma = b_alpha - 0.5 * np.log(0.06 / 0.94)
K = 16

c_bull   = 0.337
c_crit   = 0.3727
c_zero   = b_alpha / b_gamma

def pe_analytic(c):
    return K * np.sinh(2 * (b_alpha - c * b_gamma))

def b_net(c):
    return b_alpha - c * b_gamma

def theta_star_scalar(c):
    return float(expit(2 * b_net(c)))

# ── Graph generators ────────────────────────────────────────────────────────

def make_complete(N):
    A = np.ones((N, N)) - np.eye(N)
    return A / (N - 1)

def make_ring(N, k=2):
    A = np.zeros((N, N))
    for i in range(N):
        for d in range(1, k + 1):
            A[i, (i + d) % N] = 1
            A[i, (i - d) % N] = 1
    deg = A.sum(axis=1, keepdims=True)
    return A / np.where(deg == 0, 1, deg)

def make_er(N, p, rng):
    A = (rng.random((N, N)) < p).astype(float)
    A = np.triu(A, 1)
    A = A + A.T
    np.fill_diagonal(A, 0)
    deg = A.sum(axis=1, keepdims=True)
    return A / np.where(deg == 0, 1, deg)

def make_ba(N, m, rng):
    """Barabási-Albert preferential attachment, degree-normalised."""
    adj = np.zeros((N, N))
    # Seed clique
    for i in range(m + 1):
        for j in range(i + 1, m + 1):
            adj[i, j] = adj[j, i] = 1
    degrees = adj.sum(axis=0)
    for new in range(m + 1, N):
        # Only consider existing nodes 0..new-1 for attachment
        existing_deg = degrees[:new]
        total = existing_deg.sum()
        probs = existing_deg / total if total > 0 else np.ones(new) / new
        targets = rng.choice(new, size=min(m, new), replace=False, p=probs)
        for t in targets:
            adj[new, t] = adj[t, new] = 1
            degrees[new] += 1
            degrees[t]   += 1
    deg = adj.sum(axis=1, keepdims=True)
    return adj / np.where(deg == 0, 1, deg)

# ── Self-consistent mean-field solver ───────────────────────────────────────

def solve_mf(A_norm, c_vec, J, tol=1e-8, max_iter=2000):
    """
    Solve θ_i = σ(2*(b_net_i + J * Σ_j A_norm_ij * θ_j)).
    Returns θ_i array and per-node Pe vector.
    """
    bn = np.array([b_net(c) for c in c_vec])
    theta = np.array([theta_star_scalar(c) for c in c_vec])
    for _ in range(max_iter):
        field = 2 * (bn + J * A_norm @ theta)
        theta_new = expit(field)
        if np.max(np.abs(theta_new - theta)) < tol:
            theta = theta_new
            break
        theta = theta_new
    pe_vec = K * np.sinh(2 * (bn + J * A_norm @ theta))
    return theta, pe_vec

N = 24
rng = np.random.default_rng(42)

graphs = {
    "Complete":        make_complete(N),
    "Scale-free":      make_ba(N, m=2, rng=rng),
    "Random (p=0.25)": make_er(N, p=0.25, rng=rng),
    "Ring (k=2)":      make_ring(N, k=2),
}

c_uniform = np.full(N, c_bull)
J_ferro   = 0.15
J_anti    = -0.52

print(f"Network size N={N}")
print(f"Canonical: b_alpha={b_alpha:.4f}, b_gamma={b_gamma:.4f}, K={K}")
print(f"c_bull={c_bull:.4f}  c_crit={c_crit:.4f}  c_zero={c_zero:.4f}")
print(f"Single-agent Pe(c_bull) = {pe_analytic(c_bull):.3f}")
print()

results_topo = {}
print(f"{'Graph':20s}  {'deg':5s}  {'Pe_ferro':9s}  {'Pe_anti':9s}  {'ratio_ferro':12s}")
print("-" * 62)
for name, A in graphs.items():
    _, pe_f = solve_mf(A, c_uniform, J_ferro)
    _, pe_a = solve_mf(A, c_uniform, J_anti)
    raw_deg = (A > 0).sum(axis=1).mean()
    results_topo[name] = {"A": A, "pe_ferro": pe_f, "pe_anti": pe_a, "raw_deg": raw_deg}
    print(f"{name:20s}  {raw_deg:4.1f}   Pe_f={pe_f.mean():6.2f}   Pe_a={pe_a.mean():6.2f}   "
          f"{pe_f.mean()/pe_analytic(c_bull):.2f}×")

In [ ]:
# Figure 1: Topology comparison — Pe distribution per node for 4 graph types
fig, axes = plt.subplots(2, 2, figsize=(13, 9))
axes = axes.flatten()
topo_colors = {"Complete": "#2980b9", "Scale-free": "#8e44ad",
               "Random (p=0.25)": "#e67e22", "Ring (k=2)": "#27ae60"}

Pe_single = pe_analytic(c_bull)

for ax, (name, res) in zip(axes, results_topo.items()):
    pe_f = res["pe_ferro"]
    pe_a = res["pe_anti"]
    nodes = np.arange(N)
    col = topo_colors[name]

    ax.bar(nodes - 0.2, pe_f, width=0.4, color=col,       edgecolor="k", lw=0.5,
           label=f"Ferro J={J_ferro:.2f}  mean={pe_f.mean():.2f}")
    ax.bar(nodes + 0.2, pe_a, width=0.4, color=col,       edgecolor="k", lw=0.5,
           alpha=0.4, hatch="//",
           label=f"Anti  J={J_anti:.2f}  mean={pe_a.mean():.2f}")

    ax.axhline(Pe_single, color="black", linestyle="--", lw=1.3, label=f"Single-agent Pe={Pe_single:.2f}")
    ax.axhline(1.0, color="red", linestyle=":", lw=1.3, label="Pe = 1 threshold")

    deg_vec = (res["A"] > 0).sum(axis=1)
    # Colour nodes by degree
    sc = ax.scatter(nodes, pe_f, c=deg_vec, cmap="hot_r", s=40, zorder=10,
                    vmin=0, vmax=deg_vec.max())

    ax.set_title(f"{name}\n(mean degree={res['raw_deg']:.1f}, N={N})", fontsize=10)
    ax.set_xlabel("Node index", fontsize=9)
    ax.set_ylabel("Per-node Pe", fontsize=9)
    ax.legend(fontsize=7)
    ax.set_ylim(0, max(pe_f.max(), Pe_single) * 1.4)
    plt.colorbar(sc, ax=ax, label="Raw degree")

fig.suptitle(
    f"Network topology effect on Pe\n"
    f"(uniform c={c_bull:.3f}, J_ferro={J_ferro}, J_anti={J_anti},"
    f" single-agent Pe={Pe_single:.2f})",
    fontsize=11, y=1.01,
)
plt.tight_layout()
plt.savefig("nb23_topology_pe_distribution.svg", format="svg", bbox_inches="tight")
plt.show()
print("Figure 1 saved: nb23_topology_pe_distribution.svg")

In [ ]:
# Figure 2: Hub contamination
# One deep-void hub (c_hub=0.10) connected to N-1 constrained peripheral nodes
# Vary J_hub strength and c_peripheral; show Pe of peripheral nodes

N_hub = 20
c_hub_val   = 0.10     # deep void, Pe ≈ 22
c_per_vals  = np.linspace(0.32, 0.40, 40)  # sweep across c_crit=0.373
J_hub_vals  = [0.05, 0.10, 0.20, 0.30]

# Build star graph: hub=node 0, peripherals = nodes 1..N_hub-1
# Adjacency: hub connected to all, peripherals only connected to hub
def make_star_normalised(N):
    A = np.zeros((N, N))
    for i in range(1, N):
        A[0, i] = A[i, 0] = 1
    # Hub row normalised by (N-1), peripheral rows normalised by 1
    A_norm = A.copy().astype(float)
    A_norm[0, :] /= (N - 1)
    # peripheral normalisation: each peripheral has degree=1
    # already 1 per row (A[i,0]=1 for i>0)
    return A_norm

A_star = make_star_normalised(N_hub)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
ax = axes[0]

Pe_single_per_vs_c = pe_analytic(c_per_vals)
ax.plot(c_per_vals, Pe_single_per_vs_c, "k-", lw=2, label="Single-agent Pe(c)")
ax.axhline(1.0, color="red", linestyle="--", lw=1.5, label="Pe = 1 threshold")
ax.axvline(c_crit, color="red", linestyle=":", lw=1, alpha=0.7,
           label=f"c_crit = {c_crit:.4f}")

colors_j = plt.cm.plasma(np.linspace(0.1, 0.9, len(J_hub_vals)))

hub_contamination = {}
for col, J_h in zip(colors_j, J_hub_vals):
    pe_per_mf = []
    for c_per in c_per_vals:
        c_vec = np.array([c_hub_val] + [c_per] * (N_hub - 1))
        _, pe_vec = solve_mf(A_star, c_vec, J_h)
        pe_per_mf.append(pe_vec[1:].mean())  # peripheral mean Pe

    hub_contamination[J_h] = np.array(pe_per_mf)
    ax.plot(c_per_vals, pe_per_mf, color=col, lw=2,
            label=f"Network Pe_peripheral  J={J_h:.2f}")

ax.set_xlabel("Peripheral constraint $c$", fontsize=11)
ax.set_ylabel("Peripheral Pe", fontsize=11)
ax.set_title(
    f"Hub contamination: one void hub (c={c_hub_val}, Pe≈{pe_analytic(c_hub_val):.0f})\n"
    f"pulls N={N_hub-1} constrained peripherals above Pe=1",
    fontsize=10,
)
ax.legend(fontsize=8)
ax.set_xlim(c_per_vals[0], c_per_vals[-1])
ax.set_ylim(-1, 20)

# Right: contamination lift = Pe_network - Pe_single, at c_per = c_crit
ax2 = axes[1]
c_at_crit_idx = np.argmin(np.abs(c_per_vals - c_crit))
lifts = [hub_contamination[J_h][c_at_crit_idx] - 1.0 for J_h in J_hub_vals]
bars = ax2.bar([str(J) for J in J_hub_vals], lifts,
               color=colors_j, edgecolor="k", lw=0.8)
ax2.axhline(0.0, color="red", linestyle="--", lw=1.5, label="Pe_single = 1.0 (regulatory boundary)")
ax2.set_xlabel("Hub-to-peripheral coupling J", fontsize=11)
ax2.set_ylabel("Pe lift above threshold\n(at c_per = c_crit)", fontsize=11)
ax2.set_title(
    f"Hub contamination lift at c_per = c_crit\n"
    f"(hub: c={c_hub_val}, Pe≈{pe_analytic(c_hub_val):.0f})",
    fontsize=10,
)
for bar, lift in zip(bars, lifts):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
             f"+{lift:.1f}", ha="center", va="bottom", fontsize=10, fontweight="bold")
ax2.legend(fontsize=9)

plt.tight_layout()
plt.savefig("nb23_hub_contamination.svg", format="svg", bbox_inches="tight")
plt.show()
print("Figure 2 saved: nb23_hub_contamination.svg")

# Report critical J: minimum J to push Pe_peripheral > 1 when c_per = c_crit
for J_h in J_hub_vals:
    pe_at_crit = hub_contamination[J_h][c_at_crit_idx]
    print(f"  J={J_h:.2f}: Pe_peripheral(c=c_crit) = {pe_at_crit:.2f}  {'> 1 (contaminated)' if pe_at_crit > 1 else '≤ 1 (safe)'}")

In [ ]:
# Figure 3: Percolation — Pe_network vs edge density p (Erdős-Rényi)
# Show where void amplification kicks in as a function of connectivity

N_perc  = 30
p_vals  = np.linspace(0.01, 0.60, 35)
c_near_crit = c_crit + 0.002   # nodes just above c_crit (Pe ≈ 1 in isolation)
n_seeds = 8   # average over multiple random graphs

Pe_single_near = pe_analytic(c_near_crit)
print(f"Percolation experiment: N={N_perc}, c_near_crit={c_near_crit:.4f}, Pe_single={Pe_single_near:.3f}")

perc_ferro = np.zeros((len(p_vals), n_seeds))
perc_anti  = np.zeros((len(p_vals), n_seeds))

for si, seed in enumerate(range(n_seeds)):
    rng_p = np.random.default_rng(seed * 17 + 3)
    c_vec_p = np.full(N_perc, c_near_crit)
    for pi, p in enumerate(p_vals):
        A_er = make_er(N_perc, p, rng_p)
        _, pe_f = solve_mf(A_er, c_vec_p, J_ferro)
        _, pe_a = solve_mf(A_er, c_vec_p, J_anti)
        perc_ferro[pi, si] = pe_f.mean()
        perc_anti[pi, si]  = pe_a.mean()

perc_ferro_mean = perc_ferro.mean(axis=1)
perc_ferro_std  = perc_ferro.std(axis=1)
perc_anti_mean  = perc_anti.mean(axis=1)
perc_anti_std   = perc_anti.std(axis=1)

# Theoretical ER percolation threshold: p* = 1/N
p_star = 1.0 / N_perc

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax = axes[0]
ax.fill_between(p_vals, perc_ferro_mean - perc_ferro_std,
                perc_ferro_mean + perc_ferro_std, alpha=0.2, color="#e67e22")
ax.plot(p_vals, perc_ferro_mean, color="#e67e22", lw=2,
        label=f"Ferro J={J_ferro} (recommendation)")
ax.fill_between(p_vals, perc_anti_mean - perc_anti_std,
                perc_anti_mean + perc_anti_std, alpha=0.2, color="#2980b9")
ax.plot(p_vals, perc_anti_mean, color="#2980b9", lw=2,
        label=f"Anti  J={J_anti} (counter-cyclical)")

ax.axhline(Pe_single_near, color="black", linestyle="--", lw=1.3,
           label=f"Single-agent Pe={Pe_single_near:.3f}")
ax.axhline(1.0, color="red", linestyle=":", lw=1.5, label="Pe = 1 threshold")
ax.axvline(p_star, color="gray", linestyle="-.", lw=1.2, alpha=0.8,
           label=f"ER percolation p* = 1/N = {p_star:.3f}")

ax.set_xlabel("Edge density $p$", fontsize=11)
ax.set_ylabel("Mean network Pe", fontsize=11)
ax.set_title(
    f"Pe vs connectivity (Erdős-Rényi, N={N_perc})\n"
    f"Nodes at c_near_crit={c_near_crit:.4f}, single-agent Pe={Pe_single_near:.3f}",
    fontsize=10,
)
ax.legend(fontsize=8)
ax.set_xlim(0, p_vals[-1])

# Right: Pe amplification (ratio to single-agent) vs p
ax2 = axes[1]
ratio_ferro = perc_ferro_mean / Pe_single_near
ratio_anti  = perc_anti_mean  / Pe_single_near

ax2.plot(p_vals, ratio_ferro, color="#e67e22", lw=2, label="Ferro amplification")
ax2.plot(p_vals, ratio_anti,  color="#2980b9", lw=2, label="Anti suppression")
ax2.axhline(1.0, color="black", linestyle="--", lw=1.3, label="No network effect")
ax2.axvline(p_star, color="gray", linestyle="-.", lw=1.2, alpha=0.8,
            label=f"ER percolation p* = {p_star:.3f}")

# Mark where ferro ratio exceeds 2×
try:
    p_2x = p_vals[next(i for i, r in enumerate(ratio_ferro) if r > 2.0)]
    ax2.axvline(p_2x, color="#e67e22", linestyle=":", lw=1.5,
                label=f"2× amplification at p={p_2x:.3f}")
except StopIteration:
    pass

ax2.set_xlabel("Edge density $p$", fontsize=11)
ax2.set_ylabel("Pe amplification ratio (network / single)", fontsize=11)
ax2.set_title("Pe amplification grows with connectivity\n(above percolation threshold)",
              fontsize=10)
ax2.legend(fontsize=8)
ax2.set_xlim(0, p_vals[-1])
ax2.set_ylim(0, ratio_ferro.max() * 1.2)

plt.tight_layout()
plt.savefig("nb23_percolation_pe.svg", format="svg", bbox_inches="tight")
plt.show()
print("Figure 3 saved: nb23_percolation_pe.svg")

print(f"\nAmplification at p=0.50: ferro={ratio_ferro[-1]:.2f}×  anti={ratio_anti[-1]:.2f}×")

In [ ]:
# Predictions and summary stats

print("=" * 65)
print("nb23 — FALSIFIABLE PREDICTIONS")
print("=" * 65)

Pe_single_bull = pe_analytic(c_bull)

# NET-1: Scale-free has highest mean Pe (vs ring, random, complete with same mean degree)
pe_means_ferro = {n: r["pe_ferro"].mean() for n, r in results_topo.items()}
net1 = pe_means_ferro["Scale-free"] > pe_means_ferro["Ring (k=2)"]

# NET-2: Ferromagnetic coupling always raises Pe, antiferromagnetic always lowers it
net2 = all(
    r["pe_ferro"].mean() > Pe_single_bull and r["pe_anti"].mean() < Pe_single_bull
    for r in results_topo.values()
)

# NET-3: Hub contamination — even J=0.10 lifts Pe_peripheral above 1 at c_per = c_crit
net3 = hub_contamination[0.10][c_at_crit_idx] > 1.0

# NET-4: Pe amplification is monotone in p above percolation threshold
post_percolation = ratio_ferro[p_vals > p_star]
net4 = len(post_percolation) > 1 and (np.diff(post_percolation) >= 0).mean() > 0.7

predictions = [
    ("NET-1", "Scale-free network has higher mean Pe than ring (same N, same c, same J)",
     f"SF: {pe_means_ferro['Scale-free']:.2f}  Ring: {pe_means_ferro['Ring (k=2)']:.2f}", net1),
    ("NET-2", "J>0 (ferro) raises Pe above single-agent; J<0 (anti) lowers it, for all topologies",
     f"All topologies: ferro > {Pe_single_bull:.2f} > anti", net2),
    ("NET-3", "A single void hub at J≥0.10 contaminates constrained peripherals above Pe=1",
     f"J=0.10: Pe_per(c=c_crit)={hub_contamination[0.10][c_at_crit_idx]:.2f}", net3),
    ("NET-4", "Pe amplification is monotone increasing in p above ER percolation threshold",
     f"Monotone fraction = {(np.diff(post_percolation)>=0).mean():.0%}", net4),
]

all_pass = True
for pred_id, claim, metric, result_bool in predictions:
    status = "PASS" if result_bool else "FAIL"
    if not result_bool:
        all_pass = False
    print(f"\n  {pred_id}: {claim}")
    print(f"         Metric: {metric}")
    print(f"         Result: {status}")

print(f"\nAll predictions: {'PASS' if all_pass else 'PARTIAL'}")
print()
print("=" * 65)
print("KEY NUMBERS")
print("=" * 65)
print(f"  Single-agent Pe at c_bull = {Pe_single_bull:.3f}")
for name, res in results_topo.items():
    print(f"  {name:20s}: ferro Pe = {res['pe_ferro'].mean():.2f}  "
          f"anti Pe = {res['pe_anti'].mean():.2f}")
print()
print(f"  Hub (c={c_hub_val}) → peripheral (c=c_crit):")
for J_h in J_hub_vals:
    lft = hub_contamination[J_h][c_at_crit_idx]
    print(f"    J={J_h:.2f}: Pe_peripheral = {lft:.2f}")
print()
print(f"  ER amplification at p=0.50: {ratio_ferro[-1]:.2f}×")
print(f"  Anti suppression at p=0.50: {ratio_anti[-1]:.2f}×")

**Summary**

This notebook extends THRML from single agents to N-agent networks, showing that platform
topology reshapes aggregate Pe independently of individual constraint levels.

**Key results:**

1. **Topology matters.** Scale-free networks produce higher hub-node Pe than rings of equal
   mean degree, because degree-heterogeneity concentrates coupling on high-degree nodes.
   Ring topology (local coupling only) minimally perturbs single-agent Pe.

2. **Ferromagnetic vs antiferromagnetic coupling are opposite regulators.**
   - Ferro J > 0 (recommendation, algorithmic amplification): Pe > Pe_single for all topologies.
   - Anti J < 0 (counter-cyclical constraint, nb14 stablecoin coupling): Pe < Pe_single.
   Sign of J is the network analogue of the R-dimension in the void score — it sets the
   direction of collective drift.

3. **Hub contamination is the platform ecosystem risk.** A single void hub (c = 0.10,
   Pe ≈ 22) connected to constrained peripheral nodes (c ≈ c_crit) via J ≥ 0.10
   drags peripheral Pe above 1 — nodes that would individually comply become non-compliant
   through recommendation coupling alone. Regulatory compliance cannot be assessed
   platform-by-platform; the ecosystem graph must be scored.

4. **ER percolation threshold marks onset of collective void.** Pe amplification is near
   zero for p < p* = 1/N, then grows monotonically. Above p*, voids propagate across
   the network. Below p*, isolated platforms can't contaminate each other at meaningful J.

5. **Four falsifiable predictions registered:** NET-1 through NET-4.
   NET-3 is immediately testable with the 86-node void-network.json (cross-platform
   recommendation graph) — score each node's effective Pe accounting for ecosystem J.

**Three SVGs generated:**
- `nb23_topology_pe_distribution.svg` — per-node Pe for 4 topologies, ferro + anti
- `nb23_hub_contamination.svg` — void hub dragging constrained peripherals above Pe=1
- `nb23_percolation_pe.svg` — Pe amplification vs edge density, ER percolation threshold

**Next step:** Apply to the 86-node void-network.json using empirically estimated J_cross
from platform recommendation data. The network Pe map would be the first ecosystem-level
void score.

**Relates to:** `14_two_agent_contamination_critical.ipynb`, `10_cross_domain_calibration.ipynb`,
`22_regulatory_intervention_optimum.ipynb`.